In [1]:
from ows_language_model.config import config
import re

In [39]:
TEXT_PATH = config.DATASET_DIR / 'raw'
INTERIM_PATH = config.DATASET_DIR / 'interim'

In [12]:
docs = []
count = 0
for filename in TEXT_PATH.iterdir():
    with open(filename, "r") as f:
        docs.append(f.read())
        count+=1
        if count ==3000:
            break

In [13]:
start_q1 = re.compile(r"(?<=START OF THIS PROJECT GUTENBERG EBOOK).{1,150}(?=\*\*\*)")
start_q2 = re.compile(r"(?<=START OF THE PROJECT GUTENBERG EBOOK).{1,150}(?=\*\*\*)")
end_q1 = re.compile(r"end of (the|this) project gutenberg ebook")
#end_q2 = re.compile(r"(?<=START OF THE PROJECT GUTENBERG EBOOK).{1,150}(?=\*\*\*)")


In [14]:
total_names = {}
smissed = []
emissed = []
for d in docs:
    d_ = re.sub('\n', "<", d)
    ms = [f for f in re.finditer(start_q1, d_)]
    ms += [f for f in re.finditer(start_q2, d_)]
    title_start_end = []
    if len(ms):
        match = ms[0]
        name = re.sub("<", " ", match.group(0).strip())
        title_start_end += [name, match.start()]
    else:
        title_start_end += ["", 0]
        smissed.append(d)
    
    ms = [f for f in re.finditer(end_q1, d_.lower())]
    if len(ms):
        match = ms[0]
        title_start_end.append(match.start())
        total_names[name] = d[title_start_end[1]:title_start_end[2]]
    else:
        title_start_end += [-1]
        emissed.append(d)

In [15]:
len(total_names), len(docs), len(smissed), len(emissed)

(2955, 3000, 3, 1)

In [210]:
flat_text = ""
for text_ in total_names.values():
    flat_text+= re.sub(" +", " ", text_) + " <END_OF_DOC> "

In [51]:
from collections import Counter

def clean_doc(doc):
    doc = re.sub(capital_words, "", doc)
    cds = []
    for d_ in doc.split("\n\n\n\n"):
        if len(d_) > 150:
            d_ = re.sub('\-+', '-', d_)
            d_ = re.sub('\.{4,100}', '.', d_)
            d_ = re.sub('\s+', ' ', d_)
            if len(d_)>50:
                cds.append(d_)
        else:
            #print(d_)
            pass
    return cds

In [52]:
capital_words = re.compile(r"\b[A-Z\s\.]+[A-Z]\b")
english_docs = []
non_english = []
for doc_name, doc_text in total_names.items():
    most_common_word = Counter(doc_text.lower().split()).most_common()[0][0]
    if most_common_word == 'the':
        english_docs.append(clean_doc(doc_text))
    else:
        non_english.append(doc_name)

In [53]:
with open(INTERIM_PATH / 'clean_docs.json', 'w') as f:
    json.dump(english_docs, f)